In [ ]:
!pip install -U sagemaker

In [1]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()
region = sagemaker_session.boto_session.region_name

#sagemaker_session.EnableManagedSpotTraining=True

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


* 特征数据已经按天处理好

In [2]:
train_s3 = sagemaker.inputs.TrainingInput('s3://sagemaker-us-west-2-*****/cust/tf_train/features_20240725_224822/', input_mode = 'FastFile')
train_s3_1 = sagemaker.inputs.TrainingInput('s3://sagemaker-us-west-2-*****/cust/tf_train/features_20240730_035130/', input_mode = 'FastFile')
train_s3_2 = sagemaker.inputs.TrainingInput('s3://sagemaker-us-west-2-*****/cust/tf_train/features_20240730_062430/', input_mode = 'FastFile')
train_s3_3 = sagemaker.inputs.TrainingInput('s3://sagemaker-us-west-2-*****/cust/tf_train/features_20240731_233205/', input_mode = 'FastFile')
train_s3_4 = sagemaker.inputs.TrainingInput('s3://sagemaker-us-west-2-*****/cust/tf_train/features_20240801_081721/', input_mode = 'FastFile')
train_s3_5 = sagemaker.inputs.TrainingInput('s3://sagemaker-us-west-2-*****/cust/tf_train/features_20240801_095228/', input_mode = 'FastFile')
train_s3_6 = sagemaker.inputs.TrainingInput('s3://sagemaker-us-west-2-*****/cust/tf_train/features_20240801_115324/', input_mode = 'FastFile')
train_s3_7 = sagemaker.inputs.TrainingInput('s3://sagemaker-us-west-2-*****/cust/tf_train/features_20240801_143053/', input_mode = 'FastFile')

#train_s3 = 's3://liang200/cust-test-data'
#inputs = {'training': train_s3}
#print(inputs)

In [3]:
## fsx integrate

from sagemaker.inputs import FileSystemInput

# Specify FSx Lustre file system id.
file_system_id = "fs-041196881eafe09c0" # Change to your Fsx FS id

# Specify directory path for input data on the file system. 
# You need to provide normalized and absolute path below.
file_system_directory_path = '/y5pzdbev' # Change to your Fsx Mount name which is given in FSx FS details

# Specify the access mode of the mount of the directory associated with the file system. 
file_system_access_mode = 'rw'

# Specify your file system type.
file_system_type = 'FSxLustre'

fsx_fs = FileSystemInput(file_system_id=file_system_id,
                                    file_system_type=file_system_type,
                                    directory_path=file_system_directory_path,
                                    file_system_access_mode=file_system_access_mode)

#fsx_channels = {'fsx': fsx_fs}

* 特征长度json和特征唯一值json放s3，做为环境变量传入

In [ ]:
import time
timestamp = int(time.time())
original_job_name = 'job-cust-poc'
job_name = f"{original_job_name}-{timestamp}"
job_name = f"job-cust-poc-100nodes-2xlarge-7days-{timestamp}"
print(job_name)
environment = {'job_name': job_name,
               'feature_lenth_dict':'s3://sagemaker-us-west-2-*****/cust/tf_train/features_20240710_041505/part-00000-f02ce9a7-3057-4b81-b6aa-520954364de1-c000.json',
               'feature_hash_dict':'s3://sagemaker-us-west-2-*****/cust/tf_train/features_20240710_041505/part-00000-757be5db-4d45-493f-a7b9-836104157742-c000.json'
              }

In [5]:
from sagemaker.tensorflow import TensorFlow

wide_deep_estimator = TensorFlow(entry_point='./DNN-v3.py',
                             role=role,
                             source_dir = './train/',
                             #max_run=86400*3,
                             #use_spot_instances=True,
                             #max_wait=86400*3,
                             instance_count=100,
                             #train_volume_size=400,
                             #instance_type='ml.c5.18xlarge',
                             #instance_type='ml.m6i.8xlarge',
                             instance_type='ml.m6i.2xlarge',
                             #instance_type='ml.m6i.4xlarge',
                             #instance_type='ml.c6i.32xlarge',
                             #input_mode='File',
                             #input_mode='Pipe',
                             #input_mode='FastFile',
                             framework_version='2.14',
                             #py_version='py39',
                             py_version='py310',
                             subnets=['subnet-0cd82a76a056fabab'], # Should be same vpc with FSx, best to use same subnet with FSx
                             security_group_ids=['sg-0193c82932eb9f168','sg-04c9ce51b0c7665e7','sg-0af43c5507997cdb7',], # Needed when use FSx
                             #enable_sagemaker_metrics=True,
                             keep_alive_period_in_seconds=1800,
                             environment=environment,
                             distribution={'parameter_server': {'enabled': True}})

* 需要跑几天的数据，就传入几天的channel inputs
* 训练脚本中会自动扫描天级目录，如果有则加入dataset

In [ ]:
#inputs = {'fsx': fsx_fs}
#inputs = {'training': train_s3,'fsx': fsx_fs}
inputs = {'training1': train_s3_1,
          'training2': train_s3_2,
          'training3': train_s3_3,
          'training4': train_s3_4,
          'training5': train_s3_5,
          'training6': train_s3_6,
          'training7': train_s3_,
          'fsx': fsx_fs}
wide_deep_estimator.fit(inputs, job_name = job_name)